In [ ]:
###########################################################################     
############################ model training ###############################
###########################################################################


inp = input('Are you compiling on google colab?, y/n: ')
if inp.lower() == "y":
    print("We are using google colab")
    ############## if compile on google colab #################
    !pip install import-ipynb
    import import_ipynb

    !pip install -U -q PyDrive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    your_module = drive.CreateFile({'id':'1wCNx7SWrCPL78UmmzEF6pAfdniCVJUWS'})
    # https://drive.google.com/open?id=1wCNx7SWrCPL78UmmzEF6pAfdniCVJUWS
    your_module.GetContentFile('data_preprocessing.ipynb')
    import data_preprocessing as dp
    
    data = dp.process_data(1)
    
    
elif inp.lower() == "n":
    print("We are using local machine")
    ########## if compile locally ###############
    import data_preprocessing as dp
    
    data = dp.process_data(0)
    
    
else:
    print("Please reply y/n")
    
    


from fastai.vision import *
from fastai.metrics import error_rate
from sklearn.metrics import confusion_matrix

Are you compiling on google colab?, y/n: n
We are using local machine
Do you clear the folder ./original_data/?, y/n: m
Please type y/n
Do you clear the folder ./processed_data/?, y/n: n
The folder will not be cleared
Do you clear the folder ./processed_data/test?, y/n: n
The folder will not be cleared
Do you clear the folder ./processed_data/train?, y/n: n
The folder will not be cleared
Do you clear the folder ./processed_data/valid?, y/n: n
The folder will not be cleared
Do you clear the folder ./trained_models/?, y/n: n
The folder will not be cleared
Error: None - None.
./processed_data/test/
['./processed_data/train/paper/', './processed_data/train/metal/', './processed_data/train/cardboard/', './processed_data/train/trash/', './processed_data/train/glass/', './processed_data/train/plastic/']
['paper', 'metal', 'cardboard', 'trash', 'glass', 'plastic']
paper  -- size of non-trainset:  237 , size of trainset:  357
metal  -- size of non-trainset:  164 , size of trainset:  246
cardboa

/Users/admin/anaconda3/envs/tf/lib/python3.7/site-packages/fastai/vision/learner.py:105: UserWarning: `create_cnn` is deprecated and is now named `cnn_learner`.
  warn("`create_cnn` is deprecated and is now named `cnn_learner`.")


epoch,train_loss,valid_loss,error_rate,time


In [ ]:
learn = cnn_learner(data,models.resnet34,metrics=error_rate)

learn.model

learn.lr_find(start_lr=1e-6,end_lr=1e1)
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(20,max_lr=3.13e-03)

In [ ]:
preds = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
max_idxs = np.asarray(np.argmax(preds[0],axis=1))

In [ ]:
yhat = []
for max_idx in max_idxs:
    yhat.append(data.classes[max_idx])

In [ ]:
learn.data.test_ds[0][0]

In [ ]:
y = []

## convert POSIX paths to string first
for label_path in data.test_ds.items:
    y.append(str(label_path))
    
## then extract waste type from file path
pattern = re.compile("([a-z]+)[0-9]+")
for i in range(len(y)):
    y[i] = pattern.search(y[i]).group(1)

In [ ]:
## predicted values
print(yhat[0:5])
## actual values
print(y[0:5])

In [ ]:

cm = confusion_matrix(y,yhat)
print(cm)

In [ ]:
correct = 0

for r in range(len(cm)):
    for c in range(len(cm)):
        if (r==c):
            correct += cm[r,c]

In [ ]:
accuracy = correct/sum(sum(cm))
accuracy